***
# **Importación y Generación de Funciones**
***

## Librerias


In [ ]:
# Manipulacion de datos
import pandas as pd
import numpy as np
import datetime

# Graficos
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import plotly.express as px

# Seleccion de variables, modelo y metricas
import sys
import joblib
sys.modules['sklearn.externals.joblib'] = joblib
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
# Metricas (Algoritmo de clasificación)
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, roc_auc_score
# Modelos/PCA
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from scipy import stats
# K-Means
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.cluster import SilhouetteVisualizer
# Hirarchical Clustering
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.spatial.distance import pdist
from sklearn.cluster import AgglomerativeClustering as agc
# Metricas (Algoritmo de agrupación)
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import silhouette_score as sil_score

# Hiperparametros
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.model_selection import GridSearchCV

In [ ]:
!pip install mlxtend

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Para ignorar errores
import warnings
warnings.filterwarnings("ignore")
mpl.style.use('bmh')    # Diseño de los graficos

## Funciones

In [ ]:
# Funcion para crear una tabla con valores comparativos (Analisis Univariado)

def univariado_info(data):
  df_info = pd.DataFrame(columns = ['Cantidad', 'Tipo', 'Nulos', 'Unicos'])

  for col in data:
    data_series = data[col]
    df_info.loc[col] = [data_series.count(), data_series.dtype, data_series.isnull().sum(), data_series.nunique()]

  df_describe = data.describe (include = 'all', datetime_is_numeric=True).T[['top', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']]
  df_stats = pd.DataFrame([data.skew(), data.kurtosis()], index = ['sesgo', 'kurtosis']).T

  return pd.concat([df_info, pd.concat([df_describe, df_stats], axis=1)], axis=1).fillna('-')

In [ ]:
# Funcion para crear dos graficos y asi visualizar algunas variables en forma univariada:

def histogram_box_plot(data, nombre):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,8), gridspec_kw={'width_ratios': [4, 1]})

    # Histograma
    perc_25_colour = 'gold'
    perc_50_colour = 'mediumaquamarine'
    perc_75_colour = 'deepskyblue'
    perc_95_colour = 'peachpuff'

    counts, bins, patches = ax1.hist(data, facecolor=perc_50_colour, edgecolor='gray', bins='sturges')

    ax1.set_xticks(bins.round(2))
    ax1.tick_params(axis='x', rotation=70)
    ax1.set_title('Histograma con separacion por percentiles', fontsize=14)
    ax1.set_xlabel(nombre)

    q25, q75, q95 = np.percentile(data, [25, 75, 95])

    # Asignacion de colores segun barra
    for patch, leftside, rightside in zip(patches, bins[:-1], bins[1:]):
        if rightside <= q25:
            patch.set_facecolor(perc_25_colour)
        elif leftside > q95:
            patch.set_facecolor(perc_95_colour)
        elif leftside >= q75:
            patch.set_facecolor(perc_75_colour)

    bin_x_centers = 0.5 * np.diff(bins) + bins[:-1]

    # Anotaciones de cada barra en porcentaje
    for i in range(len(bins)-1):
        bin_label = "({0:,.2f}%)".format((counts[i]/counts.sum())*100)
        ax1.text(bin_x_centers[i]- 1, counts[i]*1.01, bin_label, rotation_mode='anchor')

    # Leyenda para los percentiles
    handles = [Rectangle((0,0),1,1,color=c,ec="k") for c in [perc_25_colour, perc_50_colour, perc_75_colour, perc_95_colour]]
    labels= ["0-25 Percentile","25-50 Percentile", "50-75 Percentile", ">95 Percentile"]
    ax1.legend(handles, labels, bbox_to_anchor=(0.1, 0., 0.9, 0.99))

    # Boxplot
    ax2.boxplot(data,flierprops={'markerfacecolor': 'red', 'markersize': 6}, medianprops={'color': 'green', 'linewidth': 4})
    ax2.set_xlabel(nombre)
    ax2.set_title('Boxplot', fontsize=14)

    f.tight_layout()
    plt.show()

In [ ]:
# Funcion para un Analisis Bivariado

def heatmap_plot(data):

    plt.rcParams['figure.figsize'] = 32, 15
    corr = data.corr()
    mask = np.triu(np.ones_like(corr, dtype=np.bool))
    cmap = sns.light_palette('Green', as_cmap=True)

    sns.heatmap(corr, mask=mask, cmap=cmap, annot=True, linewidth=3)

In [ ]:
# Grafico Radar para comparar entre jugadores o entre equipos

def plot_radar_compare(data, nombre1, nombre2, nombre3, nombre4, primera_columna):

    radar = data[[primera_columna, 'PTS', 'AST', 'REB', 'CHAMPION_NUM', 'VICTORY_NUM', 'FG3A', 'STL', 'BLK']]
    radar.rename(columns={
    'PTS': 'Puntos',
    'AST': 'Asistencias',
    'REB': 'Rebotes',
    'STL': 'Robos',
    'BLK': 'Bloqueos',
    'FG3A': 'Tiro de tres',
    'VICTORY_NUM': 'Partidos Ganados',
    'CHAMPION_NUM': 'Campeonatos'}, inplace=True)
    factors = ['Puntos', 'Asistencias', 'Rebotes', 'Campeonatos', 'Partidos Ganados', 'Tiro de tres', 'Robos', 'Bloqueos']

    # Definir la nueva escala de 0 a 100
    new_max = 10
    new_min = 0
    new_range = new_max - new_min

    # Transformación lineal para cada variable a [0, 10]
    for factor in factors:
        max_val = radar[factor].max()
        min_val = radar[factor].min()
        val_range = max_val - min_val
        radar[factor] = radar[factor].apply(lambda x: (((x - min_val) * new_range) / val_range) + new_min)

    radar.set_index(primera_columna, inplace=True)    # Poner la primera columna como índice

    # Graficos
    fig, axs = plt.subplots(1, 4, figsize=(26, 26), dpi=450, subplot_kw=dict(polar=True))
    nombres = [nombre1, nombre2, nombre3, nombre4]

    for i, nombre in enumerate(nombres):

      values = radar.loc[nombre].tolist()                                    # Obtener los valores del nombre seleccionado
      num_vars = len(factors)                                                # Número de variables para el gráfico.
      angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()  # Dividir el círculo en partes iguales y obtener los ángulos

      values += values[:1]
      angles += angles[:1]

      axs[i].plot(angles, values, color='black', linewidth=1)   # Graficar los datos (ángulos)
      axs[i].fill(angles, values, color='green', alpha=0.25)    # Llenar el área interior del polígono
      axs[i].set_theta_offset(np.pi / 2)                        # Ajustar los parámetros de la figura
      axs[i].set_theta_direction(-1)
      axs[i].set_rlim(0, 10)                                    # Poner las etiquetas en el eje radial
      axs[i].set_thetagrids(np.degrees(angles[:-1]), factors)   # Poner las etiquetas en el eje angular
      axs[i].set_title(nombre)                                  # Establecer el título del gráfico

    plt.show()